# Extract an individual's sparse traces and synthesised traces for illustrative purpose

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import subprocess
import sys
import yaml
import geopandas as gpd

def get_repo_root():
    """Get the root directory of the repo."""
    dir_in_repo = os.path.dirname(os.path.abspath('__file__')) # os.getcwd()
    return subprocess.check_output('git rev-parse --show-toplevel'.split(),
                                   cwd=dir_in_repo,
                                   universal_newlines=True).rstrip()
sys.path.append(get_repo_root())
ROOT_dir = get_repo_root()

import lib.helpers as helpers
import lib.models as models
import lib.saopaulo as saopaulo
import lib.genericvalidation as genericvalidation

with open(ROOT_dir + '/lib/regions.yaml') as f:
    region_manager = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
class RegionParaGenerate:
    def __init__(self, region=None):
        if not region:
            raise Exception("A valid region must be specified!")
        self.region = region
        self.path2visits = ROOT_dir + f'/dbs/{region}/visits/'
        if not os.path.exists(self.path2visits):
            os.makedirs(self.path2visits)
        self.path2geotweets = ROOT_dir + f'/dbs/{region}/geotweets.csv'
        if not os.path.exists(self.path2geotweets):
            raise Exception("The geotweets of the input region do not exist.")
        self.geotweets = None
        self.visits = None
        # Load region data
        self.region_info = region_manager[self.region]
        self.zones = None
        self.boundary = None

    def country_zones_boundary_load(self):
        # The boundary to use when removing users based on location.
        zones_loader = self.region_info['zones_loader']
        metric_epsg = self.region_info['country_metric_epsg']
        zone_id = self.region_info['country_zone_id']
        zones_path = self.region_info['country_zones_path']

        if zones_loader == 1:
            zones = gpd.read_file(ROOT_dir + zones_path)
            zones = zones.loc[zones[zone_id].notnull()]
            zones = zones.rename(columns={zone_id: "zone"})
            zones.zone = zones.zone.astype(int)
            self.zones = zones.loc[zones.geometry.notnull()].to_crs(metric_epsg)
            self.boundary = self.zones.assign(a=1).dissolve(by='a').simplify(tolerance=0.2).to_crs("EPSG:4326")

    def load_geotweets(self, only_weekday=True, only_domestic=True):
        geotweets = helpers.read_geotweets_raw(self.path2geotweets)
        if only_weekday:
            # Only look at weekday trips
            geotweets = geotweets[(geotweets['weekday'] < 6) & (0 < geotweets['weekday'])]
        # Check if keeps only domestic geotagged tweets
        if only_domestic:
            geotweets = gpd.GeoDataFrame(
                geotweets,
                crs="EPSG:4326",
                geometry=gpd.points_from_xy(geotweets.longitude, geotweets.latitude)
            )
            geotweets = gpd.clip(geotweets, self.boundary.convex_hull)
            geotweets.drop(columns=['geometry'], inplace=True)
        geotweets = geotweets.set_index('userid')
        # Remove users who don't have home visit in geotweets
        home_visits = geotweets.query("label == 'home'").groupby('userid').size()
        geotweets = geotweets.loc[home_visits.index]
        # Remove users with less than 20 tweets
        tweetcount = geotweets.groupby('userid').size()
        geotweets = geotweets.drop(labels=tweetcount[tweetcount < 20].index) # This is for domestic trip generation
        # Remove users with only one region
        regioncount = geotweets.groupby(['userid', 'region']).size().groupby('userid').size()
        geotweets = geotweets.drop(labels=regioncount[regioncount < 2].index)
        # Ensure the tweets are sorted chronologically
        self.geotweets = geotweets.sort_values(by=['userid', 'createdat'])

    def visits_gen(self, p=None, gamma=None, beta=None, days=None):
        visit_factory = models.Sampler(
            model=models.PreferentialReturn(
                p=p,
                gamma=gamma,
                region_sampling=models.RegionTransitionZipf(beta=beta, zipfs=1.2)
            ),
            n_days=days,
            daily_trips_sampling=models.NormalDistribution(mean=3.14, std=1.8)
        )
        # Calculate visits
        self.visits = visit_factory.sample(self.geotweets)
        return self.visits


## 1. Generate visits for Sao Paulo

In [4]:
p, gamma, beta = 0.89, 0.11, 0.53
days = 260
region2compute = 'saopaulo'
# prepare region data by initiating the class
print(f'{region2compute} started...')
g = RegionParaGenerate(region=region2compute)
print('Loading zones to get boundary...')
g.country_zones_boundary_load()
print('Loading geotagged tweets...')
g.load_geotweets()
print('Generating visits...')
g.visits_gen(p=p, gamma=gamma, beta=beta, days=days)

saopaulo started...
Loading zones to get boundary...
Loading geotagged tweets...
Generating visits...


,day,timeslot,kind,latitude,longitude,region
userid,,,,,,
2542,0,0,region,-23.562703,-46.697775,234
2542,0,1,region,-23.562703,-46.697775,234
2542,1,0,region,-23.562703,-46.697775,234
2542,1,1,region,-23.569502,-46.700294,242
2542,1,2,point,-23.570382,-46.696555,342
...,...,...,...,...,...,...
803997550726549504,258,0,region,-23.571815,-46.708524,6
803997550726549504,258,1,point,-23.481932,-46.706766,41
803997550726549504,258,2,point,-23.504522,-46.738174,41


In [5]:
g.geotweets.to_csv(f'../../dbs/{region2compute}/geotweets_dom_cali.csv')
g.visits.to_csv(f'../../dbs/{region2compute}/visits_dom_cali.csv')

## 2. Select an individual

In [6]:
df_users = g.geotweets.groupby('userid').size()
df_users.head()

userid
2542       447
3363        28
332923     101
550993      26
1353791     35
dtype: int64

In [15]:
# ID= 46292618
eg_id = 46292618
df_tw = g.geotweets.loc[g.geotweets.index == eg_id, :]
gdf_tw = gpd.GeoDataFrame(
    df_tw,
    crs={'init': 'epsg:4326'},
    geometry=gpd.points_from_xy(df_tw['longitude'],
                                df_tw['latitude']))

# Get spatial zones
gs = saopaulo.GroundTruthLoader()
gs.load_zones()
zones = gs.zones

# Filter out the tweets outside the study area for the sake of visualisation
gdf_tw = gpd.sjoin(gdf_tw, zones.to_crs(4326), op='intersects')
df_tw = gdf_tw.drop(columns=['geometry'])

df_visits = g.visits.loc[g.visits.index == eg_id, :]
od = genericvalidation.visits_to_odm(df_visits, zones)
od = od.reset_index()
od.columns = ['ozone', 'dzone', 'user_' + str(eg_id)]

C:\Users\yuanlia\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  app.launch_new_instance()


Convering visits to zone CRS
Aligning region-visits to zones...
removed 0 region-visits due to missing zone geom
Aligning point-visits to zones...
removed 4 point-visits due to missing zone geom
1075 visits left after alignment
Creating odm...


In [16]:
od.to_csv(ROOT_dir + '/results/input-output-example/odm.csv', index=False)
df_tw.to_csv(ROOT_dir + '/results/input-output-example/tweets.csv', index=False)